In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
from math import sqrt

In [2]:
#make sure the NYSE.txt file is in the same folder as your python script file
stocks = pd.read_csv('nyse-listed_csv.txt',delimiter="\t")

#set up our empty list to hold the stock tickers
stocks_list = []

#iterate through the pandas dataframe of tickers and append them to our empty list
for symbol in stocks['ACT Symbol,Company Name']:
    stocks_list.append(symbol)

In [3]:
len(stocks_list)

3298

In [4]:
stocks_list[:5]

['A,"Agilent Technologies, Inc. Common Stock"',
 'AA,Alcoa Inc. Common Stock',
 'AA$B,Alcoa Inc. Depository Shares Representing 1/10th Preferred Convertilble Class B Series 1',
 'AAC,"AAC Holdings, Inc. Common Stock"',
 'AAN,"Aaron\'s, Inc. Common Stock"']

In [11]:
stocks_list_test = stocks_list[:50]

In [13]:
#create empty list to hold our return series DataFrame for each stock
frames = []
 
for stock in stocks_list_test:
    
    
 
        #download stock data and place in DataFrame
        df = data.DataReader(stock, 'yahoo',start='1/1/2015')
        
        #create column to hold our 90 day rolling standard deviation
        df['Stdev'] = df['Close'].rolling(window=90).std()

        #create a column to hold our 20 day moving average
        df['Moving Average'] = df['Close'].rolling(window=20).mean()

        #create a column which holds a TRUE value if the gap down from previous day's low to next 
        #day's open is larger than the 90 day rolling standard deviation
        df['Criteria1'] = (df['Open'] - df['Low'].shift(1)) &lt; -df['Stdev'] 

        #create a column which holds a TRUE value if the opening price of the stock is above the 20 day moving average 
        df['Criteria2'] = df['Open'] &gt; df['Moving Average']

        #create a column that holds a TRUE value if both above criteria are also TRUE
        df['BUY'] = df['Criteria1'] &amp; df['Criteria2']

        #calculate daily % return series for stock
        df['Pct Change'] = (df['Close'] - df['Open']) / df['Open']

        #create a strategy return series by using the daily stock returns where the trade criteria above are met
        df['Rets'] = df['Pct Change'][df['BUY'] == True]

        #append the strategy return series to our list
        frames.append(df['Rets'])

   

KeyError: 'Date'

In [7]:
#concatenate the individual DataFrames held in our list- and do it along the column axis
masterFrame = pd.concat(frames,axis=1)

#create a column to hold the sum of all the individual daily strategy returns
masterFrame['Total'] = masterFrame.sum(axis=1)

#create a column that hold the count of the number of stocks that were traded each day
#we minus one from it so that we dont count the "Total" column we added as a trade.
masterFrame['Count'] = masterFrame.count(axis=1) - 1

#create a column that divides the "total" strategy return each day by the number of stocks traded that day to get equally weighted return.
masterFrame['Return'] = masterFrame['Total'] / masterFrame['Count']

ValueError: No objects to concatenate

In [ ]:
masterFrame['Return'].dropna().cumsum().plot()

In [ ]:
(masterFrame['Return'].mean() *252) / (masterFrame['Return'].std() * (sqrt(252)))

In [ ]:
(masterFrame['Return'].dropna().cumsum()[-1]+1)**(365.0/days) - 1

In [15]:
df

NameError: name 'df' is not defined